In [6]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
from matplotlib.pyplot import figure

In [24]:
### INSERT VARIABLES HERE ###
assetLife = 20
gearboxLife = 10
tstrike = 10
numberOfTurbines = 100 #number of turbines
inflationRate = 1.05 #Assuming 5% inflation rate

elecPriceSigma = 0.18
gearboxCost = 1 #£MM per turbine
gearboxSigma = 0.15

totalSigma = 0.085

#math.sqrt((elecPriceSigma**2)+(gearboxSigma**2))

ratedCap = 8 #MW
costPerMW = 0.9 #£1.1mil per MW
capFactor = 0.3 #Output is 30% of RC
availability = 0.95 #Assume turbines are on 95% of the time

initElecPrice = 20 #£/MWh

riskAdjRate = 0.07 #risk-adjusted discount rate
riskFreeRate = 0.03

In [25]:
### FUNCTIONS FOR BAU CASE ###

def BAUfindSetupCost(ratedCap, numberOfTurbines, costPerMW):
    setupCost = numberOfTurbines * costPerMW * ratedCap
    return setupCost

def BAUfindNetYield(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield

def BAUfindCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, netYield, tstrike):
    
    elecPrice = np.zeros(gearboxLife + tstrike) #initialise arrays
    revenue = np.zeros(gearboxLife + tstrike)
    opCosts = np.zeros(gearboxLife + tstrike)
    profit = np.zeros(gearboxLife + tstrike)
    elecPrice[0] = initElecPrice
    
    for i in range (0,assetLife):
        if(i > 0):
            elecPrice[i] = elecPrice[i-1]*inflationRate
            
        revenue[i] = netYield * elecPrice[i]
        opCosts[i] = 0.3 * revenue[i] #Assuming operation costs are 30% of revenue
        if(i >= assetLife - 3):
            profit[i] = (revenue[i] - opCosts[i]) * pow(0.8, (i - (assetLife - 4))) * pow(10, -6)
        else:
            profit[i] = (revenue[i] - opCosts[i]) * pow(10, -6) #profit in £MM
    
    return profit

def BAUfindPV(r, setupCost, assetLife, profit):
    PV = np.zeros(len(profit)) #initialise PV array
    PV[0] = -setupCost #initial cash outflow due to setup
    for i in range (1,tstrike):
        PV[i] = profit[i] / pow((1 + r), i)
        
    print("BAU Present Values:", np.around(PV, 2), '\n')

    return PV

In [26]:
### CALCULATIONS FOR BAU CASE ###

### Calculate replacement cost ###
BAUsetupCost = BAUfindSetupCost(ratedCap, numberOfTurbines, costPerMW)

### Calculate net yield ###
BAUnetYield = BAUfindNetYield(numberOfTurbines, ratedCap, capFactor, availability)

### Calculate cash in-flows ###
BAUprofit = BAUfindCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, BAUnetYield, tstrike)

### Calculate Present values ###
BAUPV = BAUfindPV(riskAdjRate, BAUsetupCost, assetLife, BAUprofit)

### Calculate NPV ###
BAUNPV = np.sum(BAUPV)
print("BAU Net Present Value = £%.2f million" % BAUNPV)

BAU Present Values: [-720.     27.44   26.93   26.42   25.93   25.44   24.97   24.5    24.04
   23.59    0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.  ] 

BAU Net Present Value = £-490.73 million


In [27]:
def findReplacementCost(numberOfTurbines, gearboxCost):
    replacementCost = gearboxCost * numberOfTurbines
    return replacementCost

In [28]:
def findNetYield(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield

In [29]:
def findCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, netYield):
    
    elecPrice = np.zeros(gearboxLife) #initialise arrays
    revenue = np.zeros(gearboxLife)
    opCosts = np.zeros(gearboxLife)
    profit = np.zeros(gearboxLife)
    elecPrice[0] = initElecPrice*pow(inflationRate,tstrike) # should move with inf rate at strike
    
    for i in range (0, gearboxLife):
        if(i>0):
            elecPrice[i] = elecPrice[i-1]*inflationRate
        
        revenue[i] = netYield * elecPrice[i]
        opCosts[i] = 0.3 * revenue[i] #Assuming operation costs are 30% of revenue
        profit[i] = (revenue[i] - opCosts[i]) * pow(10, -6) #profit in £MM
        
        #if(i >= gearboxLife - 3):
            #profit[i] = (revenue[i] - opCosts[i]) * pow(0.8, i - (gearboxLife - 4)) * pow(10, -6) #profit in £MM
            #otherwise devides by decimals due to negative power ,  which grows values
            
           
    return profit

In [30]:
def findPV(riskFreeRate, riskAdjRate, replacementCost, assetLife, gearboxLife, profit, tstrike):
    PV = np.zeros(gearboxLife) #initialise PV array
    
    #PV[0] = -replacementCost
    for i in range (1, assetLife - tstrike):
        PV[i] = profit[i] / pow((1 + riskFreeRate), tstrike+i) # should start from tstrike not tstart

    for i in range (assetLife - tstrike, gearboxLife):
        PV[i] = profit[i] / pow((1 + riskAdjRate), tstrike+i)

    print("Replacement case Present Values:", np.around(PV, 2), '\n')
    return PV

In [31]:
### CALCULATIONS FOR OPTION CASES ###

### Calculate replacement cost ###
replacementCost = findReplacementCost(numberOfTurbines, gearboxCost)

### Calculate net yield ###
netYield = findNetYield(numberOfTurbines, ratedCap, capFactor, availability)

### Calculate cash in-flows ###
profitOption = findCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, netYield)
#print(profitOption)

### Calculate Present Values ###
PVOption = findPV(riskFreeRate, riskAdjRate, replacementCost, assetLife, gearboxLife, profitOption, tstrike)

### Calculate NPVs ###
NPVOption = np.around(np.sum(PVOption), 2)
print("Net Present Value of replacement case = £%.2f million\n" % NPVOption)



Replacement case Present Values: [ 0.   34.55 35.22 35.9  36.6  37.31 38.04 38.78 39.53 40.3 ] 

Net Present Value of replacement case = £336.22 million



In [32]:
### Find Probabilities of each branch of the tree ###

u = np.exp(totalSigma * math.sqrt(1/tstrike))
d = (1/u)
u2 = u - 1
d2 = -(1-d)

p = ((1-riskFreeRate) - d2) / (u2 - d2)

print(u)
print(d)
print(p)

1.0272438687020908
0.9734786747994777
18.534692295643122


In [33]:
### Find total payoff for each branch of the tree ###

S = NPVOption
X = replacementCost / pow(inflationRate, tstrike)
print(X)

upS = S * u
downS = S * d
print(upS)
print(downS)


upValue = max(upS - X, 0)
downValue = max(downS - X, 0)
print(upValue)
print(downValue)

61.39132535407592
345.379933535017
327.3030000410804
283.9886081809411
265.9116746870045


In [34]:
### Calculate option value ###

optionValue = np.exp(-riskFreeRate * tstrike) * ((p * upValue) + ((1-p) * downValue))

totalValue = optionValue - BAUsetupCost

print("Option value = £%.2f million\n" % optionValue)
print("Total value of project = £%.2f million\n" % totalValue)

Option value = £445.20 million

Total value of project = £-274.80 million

